In [1]:
import os, sys

print("CWD:", os.getcwd())

# Añadimos la carpeta raíz del proyecto (un nivel arriba de notebooks)
root_path = os.path.abspath("..")
if root_path not in sys.path:
    sys.path.append(root_path)

root_path


CWD: C:\Users\cscpd\madly_safe\notebooks


'C:\\Users\\cscpd\\madly_safe'

In [2]:
from src.etl import cargar_y_preparar_2025

df_proc, df_target = cargar_y_preparar_2025()
df_proc.shape, df_target.shape


((32327, 25), (18136, 25))

In [3]:
df_target[["tipo_persona", "tipo_vehiculo", "rango_edad", "sexo",
           "distrito", "dia_semana", "franja_horaria"]].head()


,tipo_persona,tipo_vehiculo,rango_edad,sexo,distrito,dia_semana,franja_horaria
0,Conductor,Ciclomotor,De 30 a 34 años,Hombre,CHAMARTÍN,Miércoles,Noche_madrugada
1,Conductor,Turismo,De 40 a 44 años,Hombre,CHAMARTÍN,Miércoles,Noche_madrugada
2,Pasajero,Turismo,De 40 a 44 años,Mujer,CHAMARTÍN,Miércoles,Noche_madrugada
5,Conductor,Turismo,De 18 a 20 años,Hombre,LATINA,Miércoles,Noche_madrugada
10,Conductor,Motocicleta hasta 125cc,De 30 a 34 años,Hombre,CENTRO,Miércoles,Manana_punta


In [4]:
df_target["grave"].value_counts(normalize=True)


grave
0.0    0.983238
1.0    0.016762
Name: proportion, dtype: float64

In [5]:
!pip install scikit-learn joblib


In [6]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    roc_auc_score,
)

import joblib


In [7]:
df_target.columns


Index(['num_expediente', 'fecha', 'hora', 'localizacion', 'numero',
       'cod_distrito', 'distrito', 'tipo_accidente', 'estado_meteorológico',
       'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'sexo', 'cod_lesividad',
       'lesividad', 'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol',
       'positiva_droga', 'hora_num', 'dia_semana_num', 'dia_semana',
       'es_fin_semana', 'franja_horaria', 'grave'],
      dtype='object')

In [8]:
cols_modelo = [
    "tipo_persona",
    "tipo_vehiculo",
    "rango_edad",
    "sexo",
    "distrito",
    "dia_semana",
    "franja_horaria",
    "estado_meteorológico",  # cambia el nombre si en tu df es distinto
]

X = df_target[cols_modelo].copy()
y = df_target["grave"].astype(int)

X.head(), y.value_counts()


(   tipo_persona            tipo_vehiculo       rango_edad    sexo   distrito  \
 0     Conductor               Ciclomotor  De 30 a 34 años  Hombre  CHAMARTÍN   
 1     Conductor                  Turismo  De 40 a 44 años  Hombre  CHAMARTÍN   
 2      Pasajero                  Turismo  De 40 a 44 años   Mujer  CHAMARTÍN   
 5     Conductor                  Turismo  De 18 a 20 años  Hombre     LATINA   
 10    Conductor  Motocicleta hasta 125cc  De 30 a 34 años  Hombre     CENTRO   
 
    dia_semana   franja_horaria estado_meteorológico  
 0   Miércoles  Noche_madrugada            Despejado  
 1   Miércoles  Noche_madrugada            Despejado  
 2   Miércoles  Noche_madrugada            Despejado  
 5   Miércoles  Noche_madrugada            Despejado  
 10  Miércoles     Manana_punta            Despejado  ,
 grave
 0    17832
 1      304
 Name: count, dtype: int64)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42,
)

X_train.shape, X_test.shape, y_train.mean(), y_test.mean()


((14508, 8), (3628, 8), 0.016749379652605458, 0.01681367144432194)

In [10]:
cat_features = cols_modelo  # todas son categóricas

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, cat_features),
    ]
)

logreg = LogisticRegression(
    max_iter=500,
    class_weight="balanced",
)

modelo_baseline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("clf", logreg),
])

modelo_baseline



Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['tipo_persona',
                                                   'tipo_vehiculo',
                                                   'rango_edad', 'sexo',
                                                   'distrito', 'dia_semana',
                                                   'franja_horaria',
                                                   'estado_meteorológico'])])),
                ('clf',
                 LogisticRegression(class_weight='balanced', max_iter=500))])

In [11]:
modelo_baseline.fit(X_train, y_train)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['tipo_persona',
                                                   'tipo_vehiculo',
                                                   'rango_edad', 'sexo',
                                                   'distrito', 'dia_semana',
                                                   'franja_horaria',
                                                   'estado_meteorológico'])])),
                ('clf',
                 LogisticRegression(class_weight='balanced', max_iter=500))])

In [12]:
y_pred = modelo_baseline.predict(X_test)
y_proba = modelo_baseline.predict_proba(X_test)[:, 1]

print("Classification report:")
print(classification_report(y_test, y_pred))

print("F1 macro:", f1_score(y_test, y_pred, average="macro"))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))


Classification report:
              precision    recall  f1-score   support

           0       1.00      0.79      0.88      3567
           1       0.06      0.80      0.12        61

    accuracy                           0.79      3628
   macro avg       0.53      0.80      0.50      3628
weighted avg       0.98      0.79      0.87      3628

F1 macro: 0.4989520853611503
ROC-AUC: 0.8623608028053147
Matriz de confusión:
[[2827  740]
 [  12   49]]


In [13]:
import pathlib

ruta_modelo = pathlib.Path("..") / "models" / "modelo_logistico_2025.joblib"
ruta_modelo


WindowsPath('../models/modelo_logistico_2025.joblib')

In [14]:
joblib.dump(modelo_baseline, ruta_modelo)


['..\\models\\modelo_logistico_2025.joblib']

In [15]:
ruta_modelo.exists(), ruta_modelo


(True, WindowsPath('../models/modelo_logistico_2025.joblib'))